# DATASET

In [1]:
from qiskit_machine_learning.datasets import ad_hoc_data

adhoc_dimension = 2
train_features, train_labels, test_features, test_labels, adhoc_total = ad_hoc_data(
    training_size=10,
    test_size=1,
    n=adhoc_dimension,
    gap=0.3,
    plot_data=False,
    one_hot=False,
    include_sample_total=True,
)

# DATASET 2

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report

In [8]:
def reducir_muestras(dataset, porcentaje_mantener):
    if not 0 < porcentaje_mantener < 1:
        raise ValueError('El porcentaje a mantener debe ser un número entre 0 y 1')
    
    # Calculamos el número de muestras a mantener por cada etiqueta 'Outcome'
    num_muestras_por_etiqueta = dataset.groupby('Outcome').apply(lambda x: int(len(x) * porcentaje_mantener)).to_dict()
    
    # Seleccionamos aleatoriamente las muestras a mantener por cada etiqueta 'Outcome'
    indices = []
    for etiqueta, num_muestras in num_muestras_por_etiqueta.items():
        indices_etiqueta = np.random.choice(dataset[dataset['Outcome'] == etiqueta].index, num_muestras, replace=False)
        indices.extend(indices_etiqueta)
    
    return dataset.loc[indices]

porcentaje_mantener = 0.4

data = pd.read_csv('diabetes.csv')
data_R = reducir_muestras(data, porcentaje_mantener)

In [9]:
X = data_R.iloc[:, :-1].values
y = data_R['Outcome'].values

scaler = StandardScaler()
X = scaler.fit_transform(X)

#pca = PCA(n_components=4)
#X = pca.fit_transform(X)

train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.2, random_state=2)

# SIMULATION

In [10]:
from qiskit_ibm_runtime import Session, SamplerV2, EstimatorV2
from qiskit.providers.fake_provider import Fake127QPulseV1
from qiskit_ibm_runtime.fake_provider import FakeBrisbane 
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_aer import AerSimulator, QasmSimulator
#from qiskit_aer.primitives import SamplerV2

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = train_features.shape[1]

backend = AerSimulator(method='density_matrix', device='CPU')
session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=1, target=backend.target)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(feature_dimension=num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)

In [6]:
# Verificar si se seleccionó GPU
print(f"Backend device: {backend.options.device}")

Backend device: CPU


In [11]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=qk)

start = time.time()
qsvc.fit(train_features, train_labels)
stop = time.time()

train_time = stop - start

start = time.time()
qsvc_score = qsvc.score(test_features, test_labels)
stop = time.time()

test_time = stop - start

print(f"Training Time: {train_time}")
print(f"Testing Time: {test_time}")
print(f"Total Time: {train_time + test_time}")
print(f"QSVC classification test score: {qsvc_score}")

AlgorithmError: 'Sampler job failed!'

# QPU EXECUTION

In [9]:
from qiskit_ibm_runtime import QiskitRuntimeService

service = QiskitRuntimeService(channel="ibm_quantum", #ibm_cloud
                               token='3aa94245772e1f158c23f72db59e301c40162b1807e98fb757bc1dc9e70f9f425f48bcfd6fe16305d02d332b18eae862e3ccdb625e5d6955141bbc6ad8aa26fa')

In [10]:
# This is for saving the account so i won't need to put the token again
QiskitRuntimeService.save_account(channel='ibm_quantum',
                                  token='3aa94245772e1f158c23f72db59e301c40162b1807e98fb757bc1dc9e70f9f425f48bcfd6fe16305d02d332b18eae862e3ccdb625e5d6955141bbc6ad8aa26fa')

In [3]:
from qiskit_ibm_runtime import Session, SamplerV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService

from qiskit_machine_learning.state_fidelities import ComputeUncompute
from qiskit_machine_learning.kernels import FidelityQuantumKernel
from qiskit.circuit.library import ZZFeatureMap

num_qubits = train_features.shape[1]

service = QiskitRuntimeService()
backend = service.backend(name='ibm_brisbane')

session = Session(backend=backend)
pass_manager = generate_preset_pass_manager(optimization_level=0, backend=backend)

sampler = SamplerV2(mode=session)
fidelity = ComputeUncompute(sampler=sampler, pass_manager=pass_manager)

feature_map = ZZFeatureMap(num_qubits)
qk = FidelityQuantumKernel(feature_map=feature_map, fidelity=fidelity)


In [4]:
from qiskit_machine_learning.algorithms import QSVC
import time

qsvc = QSVC(quantum_kernel=qk)

result = qsvc.fit(train_features, train_labels)

In [5]:
start = time.time()
qsvc_score = result.score(test_features, test_labels)
stop = time.time()

test_time = stop - start

print(f"Testing Time: {test_time}")
print(f"QSVC classification test score: {qsvc_score}")

Testing Time: 59.06839895248413
QSVC classification test score: 1.0


In [1]:
from qiskit_ibm_runtime import QiskitRuntimeService
service = QiskitRuntimeService()

# Obtener el job con el ID específico
job = service.job("cydbhv77v8tg008g6m30")

In [2]:
result = job.result()